# Monte Carlo Methods

En este laboratorio vamos a explorar los métodos de Monte Carlo (Capítulo 5 del libro de Sutton y Barto). Para esto, vamos a volver a utilizar un ambiente definido en OpenAi gym (https://gym.openai.com/), es el caso esta vez de otro ambiente sencillo: el juego de Blackjack. El ambiente esta inspirado por el ejemplo 5.1 del libro, ya esta incluido en gym, por lo que no es necesario crearlo desde cero. 

***
### Juego de Blackjack

Es un juego de cartas donde el objetivo es obtener cartas que sumen lo más cercano a 21 posible, sin pasarnos. Jugamos contra un dealer fijo unicamente.

Reglas:
   
- Las cartas con figuras (Jotas, Reinas y Reyes) tienen valor de 10.
- Los Ases pueden valer 11 ó 1, cuando vale 11 se lo llama "usable".
- En este caso jugamos con un mazo infinito (con reemplazo).
- El dealer comienza con una carta boca arriba y una boca abajo.
- El jugador puede pedir una carta (HIT) hasta que decida quedarse (STICK) o exceeda los 21 puntos (BUST).
- Cuando el jugador de queda (STICK), el dealer muestra su carta boca abajo y pide cartas hasta que su suma sea 17 o más.
- Si el dealer se pasa de 21, el jugador gana. En caso contrario, gana quien tenga la suma más cerca de 21.

Ambiente:
- La reward por perder es -1, por ganar es +1 y por pedir carta es 0.
- Cada observacion es una tupla que tiene: 
    - la suma del jugador
    - la carta boca arriba del dealer (1-10 donde 1 es un As)
    - True o False si el jugador tiene un As usable o no

***

### A entregar:

- Notebook con solución a los algoritmos presentados
- Analisis de la funcion de valor estimada para distinto número de episodios a visitar (ej: 100, 1000, 10000, 50000). Se busca que puedan demostrar entendimiento del algoritmo y sus resultados (Esperamos gráficas y analisis de las mismas). 
- Estimaciones de funcion de valor para otras dos politicas definidas por el estudiante. 

In [16]:
# Para utilizar la nueva version de GYM, si es necesario reiniciar ambiente
#!pip install --upgrade gym

In [3]:
# Imports necesarios
import gym
import numpy as np
from collections import defaultdict
from tqdm import tqdm

# Creacion del ambiente
#env = gym.make('Blackjack-v0')
env = gym.make('Blackjack-v1',natural=False, sab=True)

# Acciones: 0 = Stick, 1 = HIT
print(env.observation_space)

#obs = env.reset()
obs, _ = env.reset()
print(f"Jugador: {obs[0]}, Dealer: {obs[1]}, As usable: {obs[2]} ")


#nueva_obs, reward, done, _ = env.step(0)
nueva_obs, reward, done, truncated, _ = env.step(0)

print(f"Nuevo estado: {nueva_obs}, Recompensa: {reward}, Estado Final: {done}")

Tuple(Discrete(32), Discrete(11), Discrete(2))
Jugador: 20, Dealer: 5, As usable: False 
Nuevo estado: (20, 5, False), Recompensa: 1.0, Estado Final: True


In [5]:
# Caso de prueba
#state,_ = env.reset()
state,_ = env.reset()
print('state:', state)

while True:
    action = env.action_space.sample()
    #state, reward, done, _ = env.step(action)
    state, reward, truncated, done, _ = env.step(action)
    print('state:', state, 'reward:', reward, 'done:', done, 'truncated:', truncated)
    if done or truncated:
        print(f'Reward:{reward}')                
        break

state: (18, 7, True)
state: (12, 7, False) reward: 0.0 done: False truncated: False
state: (13, 7, False) reward: 0.0 done: False truncated: False
state: (13, 7, False) reward: -1.0 done: False truncated: True
Reward:-1.0


## El algoritmo que vamos a implementar

Vamos a comenzar con el algoritmo 5.1 del libro: `First Visit MC prediction` para estimar la función de valor de una política arbitraria. 

![Image](https://marcinbogdanski.github.io/rl-sketchpad/RL_An_Introduction_2018/assets/0501_FV_MC_Pred.png)

En este caso vamos a querer evaluar una politica que se queda si el jugador tiene un valor mayor o igual a 19, y pide cartas en caso contrario. 



In [6]:
STICK = 0
HIT = 1
       
## Politica
def sample_policy(observation):
    #TODO: Implement
    pass

In [7]:
## Generar Episodio
def generate_episode(policy, env):
    states, actions, rewards = [], [], []    
    #TODO: Implement
    return states, actions, rewards

In [8]:
generate_episode(sample_policy, env)

([], [], [])

In [9]:
## Algoritmo
def first_visit_mc_prediction(policy, env, number_episodes, gamma=1):
    #TODO: Implement
    pass

In [10]:
# Uso copy popitem modifica la estructura
from copy import copy

value = first_visit_mc_prediction(sample_policy, env, number_episodes=50000)

valueaux = copy(value)
for i in range(10):
    print(valueaux.popitem())

AttributeError: 'NoneType' object has no attribute 'popitem'

In [11]:
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def plot_value_distribution(value):
    min_x = min(k[0] for k in value.keys())
    max_x = max(k[0] for k in value.keys())
    min_y = min(k[1] for k in value.keys())
    max_y = max(k[1] for k in value.keys())

    player_sum = np.arange(min_x, max_x + 1)
    dealer_show = np.arange(min_y, max_y + 1)
    X, Y = np.meshgrid(player_sum, dealer_show)

    usable_ace = np.array([False, True])
    state_values = np.zeros((len(player_sum), len(dealer_show), len(usable_ace)))
    for i, player in enumerate(player_sum):
        for j, dealer in enumerate(dealer_show):
            for k, ace in enumerate(usable_ace):
                state_values[i, j, k] = value[player, dealer, ace]
    
    fig = plt.figure()
    #ax = fig.gca(projection='3d')
    ax = fig.add_subplot(projection='3d')
    plt.title("Distribucion de valores / sin As usable")
    plt.xlabel("Suma jugador")
    plt.ylabel("Suma dealer")
    ax.view_init(ax.elev, -120)
    ax.plot_surface(X, Y, state_values[:, :, 0].T, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    plt.show()

    #Gráfico de contorno
    cp = plt.contourf(X, Y, state_values[:, :, 0].T, cmap=cm.coolwarm)
    plt.title("Contorno distribucion de valores / sin As usable")
    plt.xlabel("Suma jugador")
    plt.ylabel("Suma dealer")        
    plt.colorbar(cp)
    plt.show()

    fig = plt.figure()
    #ax = fig.gca(projection='3d')
    ax = fig.add_subplot(projection='3d')
    plt.title("Distribucion de valores / con As usable")
    plt.xlabel("Suma jugador")
    plt.ylabel("Suma dealer")
    ax.view_init(ax.elev, -120)
    ax.plot_surface(X, Y, state_values[:, :, 1].T, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    plt.show()
    
    #Gráfico de contorno
    cp = plt.contourf(X, Y, state_values[:, :, 1].T, cmap=cm.coolwarm)
    plt.title("Contorno distribucion de valores / con As usable")
    plt.xlabel("Suma jugador")
    plt.ylabel("Suma dealer")        
    plt.colorbar(cp)
    plt.show()
    

## Opcional
En esta sección presentamos las definiciones de funciones del resto de las técnicas vistas en teórico para aquellos que quieran implementarlas.

In [12]:
def first_visit_mc_prediction_Q(policy, env, number_episodes, gamma=1):
    #TODO: Implement
    """
    Los siguientes snippets pueden serle de utilidad:
        Q = defaultdict(lambda: np.zeros(env.action_space.n))
        states_actions = list(zip(states, actions))
    """
    pass

¿Observa algo extraño en los valores Q?

In [13]:
Qvalues = first_visit_mc_prediction_Q(sample_policy, env, number_episodes=50000)
Qvaluesaux = copy(Qvalues)
for i in range(10):
    print(Qvaluesaux.popitem())


TypeError: 'NoneType' object is not subscriptable

In [14]:
def first_visit_mc_prediction_Q_with_initial_exploration(policy, env, number_episodes, gamma=1):
    #TODO: Implement
    """
    Los siguientes snippets pueden serle de utilidad:
        random_action = env.action_space.sample()
        random_state = env.observation_space.sample()
    """
    pass


Antes de implementar Control MC, definiremos la clase Policy.

Esta se puede instanciar tanto para implementar:
- Una política puramente greedy (ε = 0) para Control MC **con** exploración inicial
- Una política ε-greedy para Control MC **sin** exploración inicial

In [15]:
class Policy:
    """
    Implements an epsilon-soft policy, one that returs a probability distribution for each possible action.
    
    Params
    ------
        Q: dictionary of state-action keys -> array of returns.
        number_actions: number of possible actions in the env.
        epsilon: exploration parameter where 1 explores all the time and 0 is greedy. 
    """
    
    def __init__(self, Q, number_actions, epsilon=0.1):
        self.Q = Q
        self.number_actions = number_actions
        self.epsilon = epsilon
    
    def __call__(self, observation):
        #TODO: implement
        # Se debe retornar el vector de probabilidades (página 14 del teórico).
        # Recordar, si ε = 0 -> la política es la greedy (página 13 del teórico).
        pass
    
    def update(self, new_Q):
        self.Q = new_Q

In [ ]:
def monte_carlo_control_without_initial_exploration(env, num_episodes, gamma=1, epsilon=0.1):
    #TODO: Implement
    pass